In [0]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [0]:
dbutils.fs.unmount("/mnt/inventory")


/mnt/inventory has been unmounted.


True

# Mount input container(Inventory)

In [0]:
dbutils.fs.mount(
  source = "wasbs://inventory@inventory33.blob.core.windows.net",
  mount_point = "/mnt/inventory",
  extra_configs = {"fs.azure.account.key.inventory33.blob.core.windows.net":"OyFNfd0A5TkzgjbBgldfqqEXEpAdnDYK6zTGxIZ1sebVfjY4lCTeaZhkH4d2sa9kNqMgZyU0qDR9+ASth4M68A=="})

True

#Mounting Container From ADLS for Output

In [0]:
dbutils.fs.unmount("/mnt/reports")

/mnt/reports has been unmounted.


True

In [0]:
dbutils.fs.mount(
    source="wasbs://inventory@inventory33.blob.core.windows.net/",
    mount_point="/mnt/reports",
    extra_configs={"fs.azure.account.key.inventory33.blob.core.windows.net": "OyFNfd0A5TkzgjbBgldfqqEXEpAdnDYK6zTGxIZ1sebVfjY4lCTeaZhkH4d2sa9kNqMgZyU0qDR9+ASth4M68A=="}
)


True

In [0]:
dbutils.fs.ls("/mnt")


[FileInfo(path='dbfs:/mnt/inventory/', name='inventory/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/reports/', name='reports/', size=0, modificationTime=0)]

## Read the data from Mount Point

In [0]:
df = spark.read.csv('dbfs:/mnt/inventory/Inventory_Management_Src1.csv', header=True, inferSchema=True)

In [0]:
df.write.format('parquet').mode('overwrite').save('/mnt/inventory/Inventory_management.csv',header=True)

In [0]:
df.show()

+---------+--------------+----------------+------------------+----------------+-------------+-------------+-------------+----------+-----------+------------+------------------+------------------+-------------+-----------+-------------------+-----------------+-----------------------+-------------+--------------+------------+-----------------+---------+------------------+-------------+---------------------+-----------+--------------------+----------+---------------+------------------+---------------------+
|     Date|TRANSACTION_ID|TRANSACTION_DATE|TRANSACTION_AMOUNT|TRANSACTION_TYPE|DISPATCH_DATE|EXPECTED_DATE|DELIVERY_DATE|PRODUCT_ID|CATEGORY_ID|PRODUCT Name|     PRODUCT Brand|Product Model No. |PRODUCT_STOCK|CUSTOMER_ID|      Customer Name|CUSTOMER_LOGIN_ID|CUSTOMER_STREET_ADDRESS|CUSTOMER_CITY|CUSTOMER_STATE|CUSTOMER_ZIP|CUSTOMER_PHONE_NO|SELLER_ID|       SELLER_NAME|SELLER_RATING|SELLER_STREET_ADDRESS|SELLER_CITY|        SELLER_STATE|SELLER_ZIP|SELLER_PHONE_NO|PRODUCT_COST_PRICE|PRO

In [0]:
df11 = spark.read.parquet('/mnt/inventory/Inventory_management.csv')

In [0]:
df11.show()

+---------+--------------+----------------+------------------+----------------+-------------+-------------+-------------+----------+-----------+------------+------------------+------------------+-------------+-----------+-------------------+-----------------+-----------------------+-------------+--------------+------------+-----------------+---------+------------------+-------------+---------------------+-----------+--------------------+----------+---------------+------------------+---------------------+
|     Date|TRANSACTION_ID|TRANSACTION_DATE|TRANSACTION_AMOUNT|TRANSACTION_TYPE|DISPATCH_DATE|EXPECTED_DATE|DELIVERY_DATE|PRODUCT_ID|CATEGORY_ID|PRODUCT Name|     PRODUCT Brand|Product Model No. |PRODUCT_STOCK|CUSTOMER_ID|      Customer Name|CUSTOMER_LOGIN_ID|CUSTOMER_STREET_ADDRESS|CUSTOMER_CITY|CUSTOMER_STATE|CUSTOMER_ZIP|CUSTOMER_PHONE_NO|SELLER_ID|       SELLER_NAME|SELLER_RATING|SELLER_STREET_ADDRESS|SELLER_CITY|        SELLER_STATE|SELLER_ZIP|SELLER_PHONE_NO|PRODUCT_COST_PRICE|PRO

In [0]:
customers_dim = df11.select(
    "CUSTOMER_ID", 
    "Customer Name", 
    "CUSTOMER_LOGIN_ID", 
    "CUSTOMER_STREET_ADDRESS", 
    "CUSTOMER_CITY", 
    "CUSTOMER_STATE", 
    "CUSTOMER_ZIP", 
    "CUSTOMER_PHONE_NO"
)

In [0]:
customers_dim = customers_dim.withColumnRenamed("CUSTOMER_ID", "CustomerID") \
                             .withColumnRenamed("Customer Name", "CustomerName") \
                             .withColumnRenamed("CUSTOMER_LOGIN_ID", "CustomerLoginID") \
                             .withColumnRenamed("CUSTOMER_STREET_ADDRESS", "CustomerStreetAddress") \
                             .withColumnRenamed("CUSTOMER_CITY", "CustomerCity") \
                             .withColumnRenamed("CUSTOMER_STATE", "CustomerState") \
                             .withColumnRenamed("CUSTOMER_ZIP", "CustomerZip") \
                             .withColumnRenamed("CUSTOMER_PHONE_NO", "CustomerPhoneNo")

In [0]:
customers_dim = customers_dim.dropDuplicates(["CustomerID"])

In [0]:
customers_dim.show(5)

+----------+------------------+---------------+---------------------+-------------+-------------+-----------+---------------+
|CustomerID|      CustomerName|CustomerLoginID|CustomerStreetAddress| CustomerCity|CustomerState|CustomerZip|CustomerPhoneNo|
+----------+------------------+---------------+---------------------+-------------+-------------+-----------+---------------+
|      NULL|Clinton Valenzuela|           5032|   8114 Imperdiet Av.|  Los Angeles|   California|      61854|   007-980-5252|
|    100000|    Emerald Harris|           5000| Ap #995-9477 Ulla...|     COLOUMBS|         OHIO|      70747|   023-016-5644|
|    100001|         Ulla Horn|           5001|  Ap #151-1091 Mi Av.|SAN FRANSISCO|   CALIFORNIA|      86582|   012-415-2632|
|    100002|  Chantale Gardner|           5002|         9257 Et, Rd.|  LOS ANGELES|   CALIFORNIA|      58854|   063-786-7948|
|    100003|       Fritz Grant|           5003| Ap #897-7736 Eges...| PHILADELPHIA| PENNSYLVANIA|      65342|   029-19

### Creating product DIM Table

In [0]:
products_dim = df11.select(
    "PRODUCT_ID", 
    "CATEGORY_ID", 
    "PRODUCT Name", 
    "PRODUCT Brand", 
    "`Product Model No. `", 
    "PRODUCT_STOCK"
)

In [0]:
products_dim = products_dim.withColumnRenamed("PRODUCT_ID", "ProductID") \
                           .withColumnRenamed("CATEGORY_ID", "CategoryID") \
                           .withColumnRenamed("PRODUCT Name", "ProductName") \
                           .withColumnRenamed("PRODUCT Brand", "ProductBrand") \
                           .withColumnRenamed("Product Model No. ", "ProductModelNo") \
                           .withColumnRenamed("PRODUCT_STOCK", "ProductStock")


In [0]:
products_dim = products_dim.dropDuplicates(["ProductID"])

### Output

In [0]:
products_dim.show(5)

+---------+----------+-----------+------------+--------------+------------+
|ProductID|CategoryID|ProductName|ProductBrand|ProductModelNo|ProductStock|
+---------+----------+-----------+------------+--------------+------------+
|     3000|      4000|     CAMERA|       NIKON|          1100|          10|
|     3001|      4001|      SHOES|      ADIDAS|         22234|          10|
|     3002|      4002|        BAG|        PUMA|         31240|          10|
|     3003|      4003|      WATCH|      FOSSIL|          5233|           9|
|     3004|      4004| TELEVISION|     SAMSUNG|          4170|           9|
+---------+----------+-----------+------------+--------------+------------+
only showing top 5 rows



### Creating Seller DIM table


In [0]:
seller_dim = df11.select(
    "SELLER_ID", 
    "SELLER_NAME", 
    "SELLER_RATING", 
    "SELLER_STREET_ADDRESS", 
    "SELLER_CITY", 
    "SELLER_STATE", 
    "SELLER_ZIP", 
    "SELLER_PHONE_NO"
)

In [0]:
seller_dim = seller_dim.withColumnRenamed("SELLER_ID", "SellerID") \
                       .withColumnRenamed("SELLER_NAME", "SellerName") \
                       .withColumnRenamed("SELLER_RATING", "SellerRating") \
                       .withColumnRenamed("SELLER_STREET_ADDRESS", "SellerStreetAddress") \
                       .withColumnRenamed("SELLER_CITY", "SellerCity") \
                       .withColumnRenamed("SELLER_STATE", "SellerState") \
                       .withColumnRenamed("SELLER_ZIP", "SellerZip") \
                       .withColumnRenamed("SELLER_PHONE_NO", "SellerPhoneNo")


In [0]:
seller_dim = seller_dim.dropDuplicates(["SellerID"])

### output

In [0]:
seller_dim.show(5)

+--------+------------------+------------+--------------------+----------+--------------------+---------+--------------+
|SellerID|        SellerName|SellerRating| SellerStreetAddress|SellerCity|         SellerState|SellerZip| SellerPhoneNo|
+--------+------------------+------------+--------------------+----------+--------------------+---------+--------------+
|  200000|Garrett Cunningham|           5|924-1069 Tincidun...|  GRENOBLE|         RHONE-ALPES|    83118|08 56 73 43 07|
|  200001|      Aaron Cooper|           2|P.O. Box 446, 302...|   LE MANS|    PAYS DE LA LOIRE|    94302|07 25 54 31 55|
|  200002|     Jasmine Riggs|           5|P.O. Box 832, 982...|     NIMES|LANGUEDOC-ROUSSILLON|    75377|02 43 82 62 20|
|  200003|         Hop Olson|           2|P.O. Box 969, 645...|     REIMS|   CHAMPAGNE-ARDENNE|    83495|02 73 86 40 96|
|  200004|   Nigel Macdonald|           4|  925-7953 A, Street|     TOURS|              CENTRE|    85595|02 70 13 43 17|
+--------+------------------+---

In [0]:
from pyspark.sql.functions import col, year, quarter, month, dayofweek, monotonically_increasing_id

In [0]:
df2 = spark.read.csv('dbfs:/mnt/inventory/DIM.Date.Table.csv', header=True, inferSchema=True)

In [0]:
df2.show()

+--------+-------------------+-----------+----------------+---------------+---------+----------+------------+----------------+----------------+-------------------+-------------------+--------+-----------------+----------+------------+-------+-------+------+------------+--------------+-----------+----------------------+----------------------+
|date_key|          full_date|day_of_week|day_num_in_month|day_num_overall| day_name|day_abbrev|weekday_flag|week_num_in_year|week_num_overall|    week_begin_date|week_begin_date_key|fb_month|month_num_overall|month_name|month_abbrev|quarter|fb_year|yearmo|fiscal_month|fiscal_quarter|fiscal_year|last_day_in_month_flag|same_day_year_ago_date|
+--------+-------------------+-----------+----------------+---------------+---------+----------+------------+----------------+----------------+-------------------+-------------------+--------+-----------------+----------+------------+-------+-------+------+------------+--------------+-----------+---------------

In [0]:
time_df = df11.select("TRANSACTION_DATE").dropDuplicates().withColumnRenamed("TRANSACTION_DATE", "Date")

### creating time DIM table

In [0]:
time_date_dim = df2.select(
    "fb_year", 
    "QUARTER", 
    "month_name", 
    "DAY_OF_WEEK", 
    "DATE_KEY" 
)

In [0]:
time_date_dim = time_date_dim.withColumn("TimeID", monotonically_increasing_id())

In [0]:
time_date_dim = time_date_dim.withColumnRenamed("fb_Year", "YEAR") \
                  .withColumnRenamed("Quarter", "QUARTER") \
                  .withColumnRenamed("Month_name", "MONTH") \
                  .withColumnRenamed("Day_Of_Week", "DAY_OF_WEEK") \
                  .withColumnRenamed("Date_key", "DATE")

In [0]:
time_date_dim.count()

4017

### Output


In [0]:
time_date_dim.show()

+----+-------+-------+-----------+--------+------+
|YEAR|QUARTER|  MONTH|DAY_OF_WEEK|    DATE|TimeID|
+----+-------+-------+-----------+--------+------+
|2008|      1|January|          2|20080101|     0|
|2008|      1|January|          3|20080102|     1|
|2008|      1|January|          4|20080103|     2|
|2008|      1|January|          5|20080104|     3|
|2008|      1|January|          6|20080105|     4|
|2008|      1|January|          7|20080106|     5|
|2008|      1|January|          1|20080107|     6|
|2008|      1|January|          2|20080108|     7|
|2008|      1|January|          3|20080109|     8|
|2008|      1|January|          4|20080110|     9|
|2008|      1|January|          5|20080111|    10|
|2008|      1|January|          6|20080112|    11|
|2008|      1|January|          7|20080113|    12|
|2008|      1|January|          1|20080114|    13|
|2008|      1|January|          2|20080115|    14|
|2008|      1|January|          3|20080116|    15|
|2008|      1|January|         

In [0]:
time_dim = time_df.withColumn("Year", year(col("Date"))) \
                  .withColumn("Quarter", quarter(col("Date"))) \
                  .withColumn("Month", month(col("Date"))) \
                  .withColumn("DayOfWeek", dayofweek(col("Date"))) \
                  .withColumn("TimeID", monotonically_increasing_id())


In [0]:
time_dim = time_dim.select("TimeID", "Year", "Quarter", "Month", "DayOfWeek", "Date")

In [0]:
time_dim.show(5)

+------+----+-------+-----+---------+---------+
|TimeID|Year|Quarter|Month|DayOfWeek|     Date|
+------+----+-------+-----+---------+---------+
|     0|NULL|   NULL| NULL|     NULL|28-Aug-11|
|     1|NULL|   NULL| NULL|     NULL|20-Feb-12|
|     2|NULL|   NULL| NULL|     NULL| 9-Jun-12|
|     3|NULL|   NULL| NULL|     NULL|22-Jul-12|
|     4|NULL|   NULL| NULL|     NULL|29-Jul-12|
+------+----+-------+-----+---------+---------+
only showing top 5 rows



### creating dim table

In [0]:
transaction_dim = df11.select(
    "TRANSACTION_ID", 
    "TRANSACTION_DATE", 
    "TRANSACTION_AMOUNT", 
    "TRANSACTION_TYPE", 
    "DISPATCH_DATE", 
    "EXPECTED_DATE", 
    "DELIVERY_DATE"
)

In [0]:
transaction_dim = transaction_dim.withColumnRenamed("TRANSACTION_ID", "TransactionID") \
                                     .withColumnRenamed("TRANSACTION_DATE", "TransactionDate") \
                                     .withColumnRenamed("TRANSACTION_AMOUNT", "TransactionAmount") \
                                     .withColumnRenamed("TRANSACTION_TYPE", "TransactionType") \
                                     .withColumnRenamed("DISPATCH_DATE", "DispatchDate") \
                                     .withColumnRenamed("EXPECTED_DATE", "ExpectedDate") \
                                     .withColumnRenamed("DELIVERY_DATE", "DeliveryDate")

In [0]:
transaction_dim.show()

+-------------+---------------+-----------------+---------------+------------+------------+------------+
|TransactionID|TransactionDate|TransactionAmount|TransactionType|DispatchDate|ExpectedDate|DeliveryDate|
+-------------+---------------+-----------------+---------------+------------+------------+------------+
|         5000|       1-Jan-11|             8500|    NET BANKING|    2-Jan-11|    5-Jan-11|    5-Jan-11|
|         5001|       1-Jan-11|             2450|         PAYPAL|    3-Jan-11|    6-Jan-11|    7-Jan-11|
|         5002|       1-Jan-11|             1250|            COD|    2-Jan-11|    7-Jan-11|    7-Jan-11|
|         5003|       3-Jan-11|             6750|    NET BANKING|    5-Jan-11|    8-Jan-11|    8-Jan-11|
|         5004|       5-Jan-11|            26000|    NET BANKING|    6-Jan-11|    9-Jan-11|    9-Jan-11|
|         5005|       6-Jan-11|            36000|            COD|    7-Jan-11|   10-Jan-11|   10-Jan-11|
|         5006|       7-Jan-11|            13000|      

In [0]:
inventory_fact = df11.select(
    "TRANSACTION_ID", 
    "PRODUCT_ID", 
    "CUSTOMER_ID", 
    "SELLER_ID", 
    "TRANSACTION_DATE",  
    "PRODUCT_COST_PRICE", 
    "PRODUCT_SELLING_PRICE"
)


In [0]:
inventory_fact = inventory_fact.join(
    time_dim.select("TimeID", "Date"),
    inventory_fact.TRANSACTION_DATE == time_dim.Date,
    "left"
).drop("Date", "TRANSACTION_DATE")


In [0]:
inventory_fact = inventory_fact.withColumn("FactID", monotonically_increasing_id())

In [0]:
inventory_fact = inventory_fact.select(
    "FactID", "TimeID", "TRANSACTION_ID", "PRODUCT_ID", 
    "CUSTOMER_ID", "SELLER_ID", 
    "PRODUCT_COST_PRICE", "PRODUCT_SELLING_PRICE"
)

In [0]:
inventory_fact.show(5)

+------+------+--------------+----------+-----------+---------+------------------+---------------------+
|FactID|TimeID|TRANSACTION_ID|PRODUCT_ID|CUSTOMER_ID|SELLER_ID|PRODUCT_COST_PRICE|PRODUCT_SELLING_PRICE|
+------+------+--------------+----------+-----------+---------+------------------+---------------------+
|     0|   734|          5000|      3100|     100060|   200074|             32000|                35000|
|     1|   734|          5001|      3001|     100038|   200091|              2450|                 2450|
|     2|   734|          5002|      3012|     100045|   200048|              3500|                 3500|
|     3|   322|          5003|      3089|     100090|   200055|               600|                  650|
|     4|   875|          5004|      3023|     100097|   200025|             22000|                21500|
+------+------+--------------+----------+-----------+---------+------------------+---------------------+
only showing top 5 rows



In [0]:
customers_dim.write.format('csv').mode('overwrite').save('/mnt/inventory/inventory-output/customers_dim', header=True)

In [0]:
products_dim.write.format('csv').mode('overwrite').save('/mnt/inventory/inventory-output/products_dim', header=True)

In [0]:
seller_dim.write.format('csv').mode('overwrite').save('/mnt/inventory/inventory-output/seller_dim', header=True)

In [0]:
transaction_dim.write.format('csv').mode('overwrite').save('/mnt/inventory/inventory-output/transaction_dim', header=True)

In [0]:
inventory_fact.write.format('csv').mode('overwrite').save('/mnt/inventory/inventory-output/fact_table', header=True)

In [0]:
inventory_fact = inventory_fact.withColumnRenamed("Transaction_ID", "TransactionID")

In [0]:
inventory_fact = inventory_fact.withColumnRenamed("CUSTOMER_ID", "CustomerID")

In [0]:
report_df = inventory_fact.join(
    transaction_dim.select("TransactionID", "TransactionDate"),
    on="TransactionID",
    how="inner"
).join(
    customers_dim.select("CustomerID", "CustomerName"),
    on="CustomerID",
    how="inner"
).join(
    time_date_dim.select("TimeID", "YEAR"),
    on="TIMEID",
    how="inner"
)
display(report_df)

# report_df = inventory_fact.join(
#     customers_dim.select("CustomerID", "CustomerName"),
#     inventory_fact["Customer_ID"] == customers_dim["CustomerID"],
#     how="inner"
# ).join(
#     transaction_dim.select("TransactionID", "TransactionDate"),
#     inventory_fact["TransactionID"] == transaction_dim["TransactionID"],
#     how="inner"
# )


TimeID CustomerID TransactionID FactID PRODUCT_ID SELLER_ID PRODUCT_COST_PRICE PRODUCT_SELLING_PRICE TransactionDate CustomerName YEAR 734 100060 5000 0 3100 200074 32000 35000 1-Jan-11 Vielka Rivers 2010 734 100038 5001 1 3001 200091 2450 2450 1-Jan-11 Adena Carrillo 2010 734 100045 5002 2 3012 200048 3500 3500 1-Jan-11 Darryl Ramirez 2010 322 100090 5003 3 3089 200055 600 650 3-Jan-11 Robin Mcguire 2008 875 100097 5004 4 3023 200025 22000 21500 5-Jan-11 Ignatius Mercer 2010 172 100099 5005 5 3078 200081 5600 7200 6-Jan-11 Johny Depp 2008 916 100073 5006 6 3034 200076 1600 1800 7-Jan-11 Noah Stephenson 2010 91 100081 5007 7 3067 200052 1900 1750 8-Jan-11 Rhea Lloyd 2008 91 100040 5008 8 3045 200007 3500 3200 8-Jan-11 Caryn Doyle 2008 264 100038 5009 9 3056 200000 26000 29000 10-Jan-11 Adena Carrillo 2008 578 100098 5010 10 3099 200027 8000 8000 11-Jan-11 Shaine Good 2009 37 100055 5011 11 3002 200044 1200 1250 12-Jan-11 Julie Reyes 2008 130 100095 5012 12 3013 200042 3000 3200 14-Jan-11 Harlan Zamora 2008 130 100059 5013 13 3088 200045 46000 50000 14-Jan-11 Katelyn Garrison 2008 663 100059 5014 14 3024 200054 6000 6200 15-Jan-11 Katelyn Garrison 2009 623 100062 5015 15 3077 200002 5600 5800 16-Jan-11 Christian Blackwell 2009 658 100083 5016 16 3035 200050 6500 7000 17-Jan-11 Danielle Wilcox 2009 701 100055 5017 17 3066 200082 900 1100 18-Jan-11 Julie Reyes 2009 44 100074 5018 18 3046 200007 3500 3800 19-Jan-11 Carly Parker 2008 487 100017 5019 19 3055 200095 26000 30000 20-Jan-11 Ifeoma Hale 2009 487 100051 5020 20 3098 200026 2200 2200 20-Jan-11 Montana Short 2009 921 100072 5021 21 3003 200090 6500 6750 22-Jan-11 Ayanna Riddle 2010 436 100000 5022 22 3014 200072 1200 1250 23-Jan-11 Emerald Harris 2009 265 100020 5023 23 3087 200000 16000 16000 24-Jan-11 Hu Miller 2008 822 100027 5024 24 3025 200056 8000 8500 25-Jan-11 Hedy Buchanan 2010 844 100076 5025 25 3076 200040 2100 1900 26-Jan-11 Brennan Sweet 2010 298 100039 5026 26 3036 200081 2100 2300 28-Jan-11 Jacob Clark 2008 298 100072 5027 27 3065 200024 500 550 28-Jan-11 Ayanna Riddle 2008 298 100019 5028 28 3047 200062 18000 19000 28-Jan-11 Gary Burt 2008 702 100065 5029 29 3054 200061 400 600 30-Jan-11 Colleen Church 2009 902 100088 5030 30 3006 200055 13000 13000 31-Jan-11 Gisela Mayer 2010 88 100021 5031 31 3093 200019 47000 52000 1-Feb-11 Nora Stuart 2008 478 100042 5032 32 3015 200077 1400 1400 2-Feb-11 Leroy Pugh 2009 478 100075 5033 33 3086 200082 28000 31000 2-Feb-11 Libby Dyer 2009 478 100057 5034 34 3022 200030 32000 34000 2-Feb-11 Edward Lopez 2009 299 100068 5035 35 3075 200095 450 500 5-Feb-11 Tana Blevins 2008 739 100039 5036 36 3039 200068 600 650 6-Feb-11 Jacob Clark 2010 558 100061 5037 37 3069 200005 750 650 7-Feb-11 Brett Harper 2009 876 100052 5038 38 3044 200013 3200 3600 8-Feb-11 Oprah Chaney 2010 876 100070 5039 39 3052 200046 5000 4800 8-Feb-11 Igor Lucas 2010 927 100061 5040 40 3008 200054 600 650 10-Feb-11 Brett Harper 2010 205 100084 5041 41 3097 200012 3600 3800 11-Feb-11 Ebony Durham 2008 758 100056 5042 42 3016 200091 1200 1350 12-Feb-11 Fletcher Chang 2010 373 100087 5043 43 3084 200077 660 800 13-Feb-11 Lawrence Salinas 2009 826 100074 5044 44 3026 200012 38000 41000 15-Feb-11 Carly Parker 2010 826 100046 5045 45 3074 200006 600 500 15-Feb-11 Orlando Richmond 2010 151 100059 5046 46 3031 200080 4000 4500 16-Feb-11 Katelyn Garrison 2008 775 100063 5047 47 3062 200095 1800 1600 18-Feb-11 Scarlet Burks 2010 775 100095 5048 48 3042 200093 600 650 18-Feb-11 Harlan Zamora 2010 28 100003 5049 49 3051 200021 16000 16500 19-Feb-11 Fritz Grant 2008 877 100046 5050 50 3009 200068 42000 42000 20-Feb-11 Orlando Richmond 2010 616 100073 5051 51 3094 200009 560 750 21-Feb-11 Noah Stephenson 2009 115 100009 5052 52 3011 200028 2500 2650 22-Feb-11 Stacy Shaffer 2008 49 100100 5053 53 3081 200088 800 650 23-Feb-11 Will Smith 2008 713 100094 5054 54 3021 200016 1400 1460 24-Feb-11 Claudia Carter 2009 593 100078 5055 55 3072 200074 5600 5800 25-Feb-11 Caldwell W

In [0]:
from pyspark.sql.functions import year, col, sum
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

In [0]:
aggregated_df = report_df.groupBy("YEAR", "CustomerName") \
    .agg(sum("PRODUCT_SELLING_PRICE").alias("TotalSellingPrice"))

In [0]:
# aggregated_df = report_df.withColumn("Year", year(col("TransactionDate"))) \
#     .groupBy("Year", "CustomerName") \
#     .agg(sum("PRODUCT_SELLING_PRICE").alias("TotalSellingPrice"))

In [0]:
window_spec = Window.partitionBy("Year").orderBy(col("TotalSellingPrice").desc())
ranked_df = aggregated_df.withColumn("Rank", rank().over(window_spec))

In [0]:
top10_customers = ranked_df.filter(col("Rank") <= 10).select("Year", "CustomerName", "TotalSellingPrice")

In [0]:
top10_customers.show(10)

+----+------------------+-----------------+
|Year|      CustomerName|TotalSellingPrice|
+----+------------------+-----------------+
|2008|  Katelyn Garrison|           218100|
|2008|       Julie Reyes|           143450|
|2008|      Carly Parker|           127000|
|2008|    Adena Carrillo|           123050|
|2008|       Nora Stuart|           120100|
|2008|       Echo Spence|           115800|
|2008|        Rhea Lloyd|           115750|
|2008|     Charles Kline|           108550|
|2008|Clinton Valenzuela|            88250|
|2008|  Orlando Richmond|            85650|
+----+------------------+-----------------+
only showing top 10 rows



In [0]:
state_orders = inventory_fact.join(
    customers_dim.select("CustomerID", "CustomerState"),
    on="CustomerID",
    how="inner"
)

In [0]:
from pyspark.sql.functions import count

In [0]:
statewise_ordercount = state_orders.groupBy("CustomerState") \
    .agg(count("TransactionID").alias("NumberOfOrders")) \
    .orderBy(col("NumberOfOrders").desc())

In [0]:
statewise_ordercount.show()

+--------------------+--------------+
|       CustomerState|NumberOfOrders|
+--------------------+--------------+
|               TEXAS|           230|
|          CALIFORNIA|           194|
|          California|           103|
|                OHIO|           101|
|             ARIZONA|            78|
|      MASSAACHUSETTS|            72|
|            ILLINOIS|            64|
|        PENNSYLVANIA|            62|
|             NEWYORK|            51|
|DISTRICT OF COLUMBIA|            44|
+--------------------+--------------+



In [0]:
preferred_payment_mode = transaction_dim.groupBy("TransactionType") \
    .agg(count("TransactionID").alias("OrderCount")) \
    .orderBy(col("OrderCount").desc())

In [0]:
preferred_payment_mode.show()

+---------------+----------+
|TransactionType|OrderCount|
+---------------+----------+
|    NET BANKING|       267|
|            COD|       248|
|         PAYPAL|       199|
|     DEBIT CARD|       178|
|    CREDIT CARD|       108|
+---------------+----------+



In [0]:
report2_df = inventory_fact.join(
    time_date_dim.select("TimeID", "YEAR", "QUARTER"),
    on="TIMEID",
    how="inner"
)

In [0]:
profit_report = report2_df.withColumn(
    "Profit", col("PRODUCT_SELLING_PRICE") - col("PRODUCT_COST_PRICE")
).groupBy("YEAR", "QUARTER") \
    .agg(sum("Profit").alias("TotalProfit")) \
    .orderBy("YEAR", "QUARTER")

In [0]:
yearly_profit = profit_report.filter(col("YEAR") == 2011)

In [0]:
yearly_profit.show(truncate=False)

+----+-------+-----------+
|YEAR|QUARTER|TotalProfit|
+----+-------+-----------+
+----+-------+-----------+



In [0]:
products_dim = products_dim.withColumnRenamed("ProductID", "PRODUCT_ID")

In [0]:
report3_df = inventory_fact.join(
    time_date_dim.select("TimeID", "YEAR", "QUARTER"),
    on="TIMEID",
    how="inner"
).join(
    products_dim.select("PRODUCT_ID", "CATEGORYID", "PRODUCTNAME"),
    on="PRODUCT_ID",
    how="inner"
)

In [0]:
sales_count = report3_df.groupBy("YEAR", "QUARTER", "CATEGORYID", "PRODUCTNAME") \
    .agg(count("PRODUCT_ID").alias("SalesCount")) \
    .orderBy("YEAR", "QUARTER", "CATEGORYID")

In [0]:
# sales_count = report3_df.groupBy("YEAR", "PRODUCTNAME", "QUARTER") \
#     .agg(count("PRODUCT_ID").alias("SalesCount")) \
#     .orderBy("YEAR", "PRODUCTNAME", "QUARTER")
sales_count = report3_df.groupBy("YEAR", "CATEGORYID", "PRODUCTNAME", "QUARTER") \
    .agg(count("PRODUCT_ID").alias("SalesCount")) \
    .orderBy("YEAR", "CATEGORYID", "PRODUCTNAME", "QUARTER")


In [0]:
sales_count.count()

120

In [0]:
sales_count.show(8)

+----+----------+-----------+-------+----------+
|YEAR|CATEGORYID|PRODUCTNAME|QUARTER|SalesCount|
+----+----------+-----------+-------+----------+
|2008|      4000|     CAMERA|      1|        10|
|2008|      4000|     CAMERA|      2|         4|
|2008|      4000|     CAMERA|      3|         7|
|2008|      4000|     CAMERA|      4|         4|
|2008|      4001|      SHOES|      1|        11|
|2008|      4001|      SHOES|      2|        10|
|2008|      4001|      SHOES|      3|        10|
|2008|      4001|      SHOES|      4|         8|
+----+----------+-----------+-------+----------+
only showing top 8 rows



In [0]:
top10_customers.write.csv("/mnt/reports/top10_customers.csv", mode="overwrite", header=True)
statewise_ordercount.write.csv("/mnt/reports/statewise_ordercount.csv", mode="overwrite", header=True)
preferred_payment_mode.write.csv("/mnt/reports/preferred_payment_mode.csv", mode="overwrite", header=True)
yearly_profit.write.csv("/mnt/reports/yearly_profit.csv", mode="overwrite", header=True)
sales_count.write.csv("/mnt/reports/sales_count.csv", mode="overwrite", header=True)


In [0]:
dbutils.fs.ls("/mnt/reports")

[FileInfo(path='dbfs:/mnt/reports/DIM.Date.Table.csv', name='DIM.Date.Table.csv', size=796835, modificationTime=1733380224000),
 FileInfo(path='dbfs:/mnt/reports/Inventory_Management_Src1.csv', name='Inventory_Management_Src1.csv', size=287957, modificationTime=1733122475000),
 FileInfo(path='dbfs:/mnt/reports/Inventory_management.csv/', name='Inventory_management.csv/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/reports/inventory-output/', name='inventory-output/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/reports/preferred_payment_mode.csv/', name='preferred_payment_mode.csv/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/reports/sales_count.csv/', name='sales_count.csv/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/reports/statewise_ordercount.csv/', name='statewise_ordercount.csv/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/reports/top10_customers.csv/', name='top10_customers.csv/', size=0, modificationTime=0),
 FileInfo(pa

In [0]:
jdbc_url = "jdbc:sqlserver://inventorymanagement.database.windows.net:1433;database=inventorymanagement"
connection_properties = {
    "user": "inventorymanagement",
    "password": "Inventory@33",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}


In [0]:
# Save Top_customers to Azure SQL Database
top10_customers.write.jdbc(
    url=jdbc_url, 
    table="Top10_customers", 
    mode="overwrite", 
    properties=connection_properties
)

# Save statewise_ordercount to Azure SQL Database
statewise_ordercount.write.jdbc(
    url=jdbc_url, 
    table="statewise_ordercount", 
    mode="overwrite", 
    properties=connection_properties
)

# Save preferred_payment_mode to Azure SQL Database
preferred_payment_mode.write.jdbc(
    url=jdbc_url, 
    table="preferred_payment_mode", 
    mode="overwrite", 
    properties=connection_properties
)

# Save yearly_profit to Azure SQL Database
yearly_profit.write.jdbc(
    url=jdbc_url, 
    table="yearly_profit", 
    mode="overwrite", 
    properties=connection_properties
)

# Save sales_count to Azure SQL Database
sales_count.write.jdbc(
    url=jdbc_url, 
    table="sales_count", 
    mode="overwrite", 
    properties=connection_properties
)
